In [53]:
from skyfield.api import load
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from skyfield.api import utc
from scipy.optimize import brentq # machine learning

from datetime import timedelta, datetime
import pytz

In [18]:
# load Ephemeris
# load library planet position file
planets = load('de421.bsp')

_planets = {
    "sun": planets['SUN'],
    "earth": planets['EARTH'],
    "moon": planets['MOON'],
    "mercury": planets['MERCURY'],
    "venus": planets['VENUS BARYCENTER'],
    "mars": planets['MARS BARYCENTER'],
    "jupiter": planets['JUPITER BARYCENTER'],
    "saturn": planets['SATURN BARYCENTER'],
    "uranus": planets['URANUS BARYCENTER'],
    "neptune": planets['NEPTUNE BARYCENTER'],
    "pluto": planets['PLUTO BARYCENTER'],
}

In [19]:
ts = load.timescale(builtin=True)

def get_ts(birthday_datetime):
    return ts.utc(birthday_datetime)

In [70]:
def planet1_looks_at_planet2(p1, p2, t):
    return p1.at(t).observe(p2).apparent()

def observe(p1, p2, t):
    """ Observe planet 1 to planet 2 ... observe(p1, p2, timestamp).degrees, or use .radians for radiance """
    t = get_ts(t)

    astrometric = planet1_looks_at_planet2(p1, p2, t)
    _, lon, _ = astrometric.ecliptic_latlon()
    return lon

def getEarthPos(t, planet):
    """ Given a time t and planet p, return longitude position of planet on ecliptic plane """

    return observe(_planets['earth'], planet, t).radians

In [71]:
t = datetime(1970, 12, 9, 0, 14, 0, tzinfo=utc)
earth = _planets['earth']
jupiter = _planets['jupiter']
observe(earth, jupiter, t).radians

4.072675942197303

## Generate a whole set of dates, spaced every 2H

In [73]:
start = datetime(1970, 1, 1, 0, 0, 0, tzinfo=utc)
end = datetime(2020, 4, 12, 0, 0, 0, tzinfo=utc)
dates = pd.date_range(start = start, end = end, tz=pytz.UTC, freq='2H').to_pydatetime().tolist()

# reason for 2H frequency: we want to divide the day up in 12 segments, which is a nice amount
len(dates)

220369

In [75]:
import math

In [102]:
bodies = ['sun', 'earth', 'moon', 'venus', 'mars', 'jupiter']

positions = {
    'sun': (lambda t: observe(earth,      _planets['sun'], t).radians),
    'earth': (lambda t: observe(earth,    _planets['sun'], t).radians - math.pi),
    'moon': (lambda t: observe(earth,     _planets['moon'], t).radians),
    'venus': (lambda t: observe(earth,  _planets['venus'], t).radians),
    'mars': (lambda t: observe(earth,  _planets['mars'], t).radians),
    'jupiter': (lambda t: observe(earth,    _planets['jupiter'], t).radians)
}

def generate_table(dates):

    new_table = {}
    new_table['t'] = dates

    for b in bodies:
        new_table[b] = [] # assign body nae as key
        for t in dates: # calculate for every date in dates
            body_pos = positions[b](t) # calculate position, using this planet of body
            new_table[b].append( body_pos ) # generate a new table

    df = pd.DataFrame(data=new_table)

    return df

df = generate_table(dates)
df.sample(15)

,t,sun,earth,moon,venus,mars,jupiter
149690,2004-02-26 04:00:00+00:00,5.878904,2.737311,0.791900,0.353795,0.776064,2.876731
125890,1998-09-21 20:00:00+00:00,3.118022,-0.023571,3.334024,2.943443,2.449233,6.149105
73922,1986-11-13 04:00:00+00:00,4.025802,0.884210,0.223293,3.812943,5.612511,5.989845
67042,1985-04-18 20:00:00+00:00,0.504923,-2.636670,0.242316,0.121726,0.958782,5.477937
4485,1971-01-09 18:00:00+00:00,5.048080,1.906488,1.539050,4.238912,4.049131,4.181739
160677,2006-08-29 18:00:00+00:00,2.725605,-0.415987,3.856335,2.454635,3.034209,3.891856
92640,1991-02-20 00:00:00+00:00,5.776359,2.634766,0.621809,6.234074,1.220381,2.198069
182368,2011-08-11 08:00:00+00:00,2.411440,-0.730153,5.047272,2.386536,1.660161,0.691000
52811,1982-01-18 22:00:00+00:00,5.214145,2.072552,4.012284,5.285399,3.382649,3.815142
91068,1990-10-12 00:00:00+00:00,3.465056,0.323463,2.083998,3.372365,1.293873,2.270147


In [104]:
df.to_csv('planets.csv', sep=';')